In [ ]:
# -*- coding: utf-8 -*-
"""
Script para cálculo de evapotranspiração real (ETr) usando o modelo SSEBop
Autor: Raphael Casari (modificado de Leandro Salles)
Data: 21/11/2019 (última modificação)
"""

# ==================== IMPORTAÇÃO DE BIBLIOTECAS ====================
import numpy as np
import os
import matplotlib.pyplot as plt
import ETo_FAL as ETo_FAL_script
import rasterio_and_xarray_SSEBOp as raxa
import gdal_and_xarray as gdal_xa

# ==================== CONFIGURAÇÃO DE PATHS ====================
# Diretórios principais
BASE_DIR = 'C:/Users/Raphael Casari/Desktop/Processamento-evapo'
PATH_MET = os.path.join(BASE_DIR, 'Meteorologia')
PATH_NDVI = os.path.join(BASE_DIR, 'Images/NDVI')
PATH_LST = os.path.join(BASE_DIR, 'Images/Thermal')

# Arquivos de entrada
MET_FILE = 'Dados_meteorologicos_2019_atualizado.xlsx'
MET_SHEET = 'Planilha1'
NDVI_FILE = '2019-08-21-NDVI.tif'
LST_FILE = '2019-08-21-Thermal.tif'

# ==================== CARREGAMENTO DE DADOS ====================
# Dados meteorológicos
dados_meteorologicos = ETo_FAL_script.estacao_FAL(PATH_MET, MET_FILE, MET_SHEET)

# Extração de variáveis para a data específica (2019-08-21)
Tmax_C = dados_meteorologicos['Tmax'].loc['2019-08-21']
Tmax_K = dados_meteorologicos['Tmax_K'].loc['2019-08-21']
dT = dados_meteorologicos['dT_v2013'].loc['2019-08-21']
dT_2018 = dados_meteorologicos['dT_v2018'].loc['2019-08-21']
ETo = dados_meteorologicos['ETo'].loc['2019-08-21']
print(f"ETo calculada: {ETo}")

# Carregamento de imagens NDVI e LST
ndvi_xa = raxa.rasterio_to_xarray(os.path.join(PATH_NDVI, NDVI_FILE))
LST_Celsius = raxa.rasterio_to_xarray(os.path.join(PATH_LST, LST_FILE))
LST_xa = LST_Celsius + 273.16  # Conversão para Kelvin

# ==================== PRÉ-PROCESSAMENTO ====================
# Reindexação para alinhar NDVI e LST
ndvi_idxLST = ndvi_xa.reindex(
    latitude=LST_xa.latitude.values,
    longitude=LST_xa.longitude.values,
    method='nearest'
)

# Salvamento dos arquivos reindexados
suffix_ndvi = 'idxLST_' + NDVI_FILE[:-4]
suffix_LST = 'idxLST_' + LST_FILE[:-4]

NDV, xsize, ysize, GeoT, Projection, data = gdal_xa.GetTiffInfobyName(os.path.join(PATH_LST, LST_FILE), ndvi_idxLST)
gdal_xa.create_geotiff(suffix_ndvi, data, np.nan, xsize, ysize, GeoT, Projection)

# Máscara para valores NDV
ndvi = np.ma.masked_array(ndvi_idxLST, mask=ndvi_idxLST == NDV, fill_value=np.nan)
LST = np.ma.masked_array(LST_xa, mask=LST_xa == NDV, fill_value=np.nan)

# ==================== CÁLCULO DO FATOR DE CORREÇÃO (c) ====================
# Cálculo de diferença de temperatura e razão LST/Tmax
Tdiff = Tmax_K - LST
Tcorr = LST / Tmax_K

# Filtro para pixels válidos
v = np.where(
    (ndvi >= 0.8) & (ndvi < 1) &
    (LST > 270) &
    (Tdiff >= 0) & (Tdiff <= 10)
)
Tcorr2 = Tcorr[v]

# Histograma para análise
plt.hist(Tcorr2, 50, density=True, facecolor='g', alpha=0.75)
plt.show()

# Cálculo do fator de correção (2 desvios padrão abaixo da média)
media = np.mean(Tcorr2)
std = np.std(Tcorr2)
c = media - (2 * std)
print(f"Fator de correção (c): {c}")

# ==================== CÁLCULO SSEBop (2013) ====================
K = 1  # Coeficiente de ajuste

# Temperaturas quente e fria
Tc = c * Tmax_K
Th = Tc + dT

# Fração Evapotranspirativa (ETf)
ETf = (Th - LST) / dT
ETf[ETf < 0] = np.nan
ETf[(ETf > 1.05) & (ETf <= 1.30)] = 1.05
ETf[ETf > 1.30] = np.nan

# Evapotranspiração Real (ETr)
ETr = ETf * ETo * K

# ==================== SALVAMENTO DE RESULTADOS ====================
NDV, xsize, ysize, GeoT, Projection, data = gdal_xa.GetTiffInfobyName(os.path.join(PATH_LST, LST_FILE), ETr)
suffix_ETr = 'ETa_' + LST_FILE[:-4]
suffix_ETf = 'ETf_' + LST_FILE[:-4]
gdal_xa.create_geotiff(suffix_ETr, ETr, NDV, xsize, ysize, GeoT, Projection)
gdal_xa.create_geotiff(suffix_ETf, ETf, NDV, xsize, ysize, GeoT, Projection)
